# Grid Explorer

This example allows you to interactively explore the spectra in a `synthesizer` grid.

In [ ]:
import numpy as np
from synthesizer.grid import Grid
from synthesizer.sed import plot_spectra

try:
    import ipywidgets as widgets
except ImportError:
    import sys

    print("ipywidgets not found. Please install with pip install ipywidgets.")
    sys.exit(0)

In [ ]:
grid_dir = "../../tests/test_grid/"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

print(grid)

In [ ]:
def update_spectra(
    log10age_grid_point=0,
    metallicity_grid_point=0,
    spectra_id="incident",
    wavelength_range=[2.5, 4.0],
):
    """
    Function to make a plot of the spectra.

    Arguments:
        log10age_grid_point (int)
            The age grid point.
        metallicity_grid_point (int)
            The metallicity grid point.
        spectra_id (str)
            The id of the spectra to be plotted.
        wavelength_range (list, float)
            The log10(wavelength/Angstrom) range to plot
    """

    # Define grid point - set by widget
    grid_point = (log10age_grid_point, metallicity_grid_point)

    # Get sed object - spectra_id set by widget
    sed = grid.get_spectra(grid_point, spectra_id=spectra_id)

    # Alternative implementation using plot_spectra
    fig, ax = plot_spectra(
        sed,
        xlimits=10 ** np.array(wavelength_range),
        show=True,
        label=spectra_id,
    )

    # Add labels for age and metallicity
    fig.text(
        0.15,
        0.2,
        rf"$\log_{{10}}(age/Myr): {grid.log10age[log10age_grid_point]:.2f}$",
    )
    fig.text(0.15, 0.15, rf"$Z: {grid.metallicity[metallicity_grid_point]}$")


# Spectra selection widget
spectra_id = widgets.Dropdown(
    options=["incident", "nebular", "total"],
    value="incident",
    description="spectra type",
    disabled=False,
)

# Wavelength range selection widget
wavelength_range = widgets.FloatRangeSlider(
    value=[2.5, 4.0],
    min=0,
    max=6.0,
    step=0.1,
    description="Wavelength range:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)

# Age index selection widget
age = widgets.IntSlider(
    min=0, max=len(grid.log10age) - 1, step=1, value=0, description="age index"
)

# Metallicity index selection widget
metallicity = widgets.IntSlider(
    min=0,
    max=len(grid.metallicity) - 1,
    step=1,
    value=0,
    description="metallicity index",
)

# Define UI
ui = widgets.VBox([wavelength_range, spectra_id, age, metallicity])

# Collect widgets
out = widgets.interactive_output(
    update_spectra,
    {
        "log10age_grid_point": age,
        "metallicity_grid_point": metallicity,
        "spectra_id": spectra_id,
        "wavelength_range": wavelength_range,
    },
)

# Display
display(ui, out)